In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

while 'notebooks' in os.getcwd():
    os.chdir('..')

import pandas as pd
import torch
import torch.nn.functional as F
from torch_sparse.tensor import SparseTensor
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from sklearn.metrics import roc_auc_score
import logging
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import numpy as np

from src.torch_geo_models import GCN
from src.data.node_classifier.arxiv import load_dataset_pyg, data_to_sparse_symmetric_pyg

In [3]:
logging.basicConfig(
    format='%(asctime)s - %(levelname)s : %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [4]:
torch.cuda.is_available()

True

In [5]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

## Data Loading

In [6]:
dataset = load_dataset_pyg()

In [7]:
data = data_to_sparse_symmetric_pyg(dataset[0])
data.adj_t = data.adj_t.to(device)

In [8]:
split_idx = dataset.get_idx_split()

In [9]:
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

## Create `edge_weights`

In [10]:
edges_gamma_df = pd.read_csv('data/gamma_graph_sage/01-1_gamma_scored_edges.csv')
print(edges_gamma_df.shape)
edges_gamma_df.head()

(2315598, 3)


,source,target,gamma_pred
0,411,0,0.649669
1,640,0,0.784250
2,1162,0,0.787807
3,1897,0,0.787201
4,3396,0,0.784839


In [11]:
edge_weights = SparseTensor.from_torch_sparse_coo_tensor(
    torch.sparse_coo_tensor(
        edges_gamma_df[['source', 'target']].values.T,
        edges_gamma_df['gamma_pred'].values,
        size=(data.adj_t.size(0), data.adj_t.size(1))))\
    .to(device)

In [12]:
edge_weights.is_symmetric()

True

In [13]:
samp = edges_gamma_df.sample(1).iloc[0]
s, t = int(samp['source']), int(samp['target'])

(edge_weights[s, t], edge_weights[t, s])

(SparseTensor(row=tensor([0], device='cuda:0'),
              col=tensor([0], device='cuda:0'),
              val=tensor([0.7797], device='cuda:0', dtype=torch.float64),
              size=(1, 1), nnz=1, density=100.00%),
 SparseTensor(row=tensor([0], device='cuda:0'),
              col=tensor([0], device='cuda:0'),
              val=tensor([0.7797], device='cuda:0', dtype=torch.float64),
              size=(1, 1), nnz=1, density=100.00%))

### Model

In [14]:
features = data.x.cuda()
labels = data.y.cuda()
train_mask = split_idx['train'].cuda()
val_mask = split_idx['valid'].cuda()
test_mask = split_idx['test'].cuda()

In [15]:
train_nid = train_mask.nonzero().squeeze()
val_nid = val_mask.nonzero().squeeze()
test_nid = test_mask.nonzero().squeeze()

In [16]:
n_layers = 3
n_iters = 3000
epochs = 5000
log_steps = 100
input_dim = features.shape[1]
hidden_channels = input_dim * 2
output_dim = dataset.num_classes
lr_rate = 0.001
dropout = 0.5

In [17]:
model = GCN(
    n_layers=n_layers,
    in_channels=input_dim,
    hidden_channels=hidden_channels,
    out_channels=output_dim,
    dropout=dropout,
    batch_norm=True)\
    .to(device)

In [18]:
def train(model, graph, features, train_mask, optimizer, edge_weight=None):
    model.train()

    optimizer.zero_grad()
    out = model(features, graph, edge_weight=edge_weight)[train_mask]
    loss = F.nll_loss(out, labels.squeeze(1)[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()

In [19]:
@torch.no_grad()
def test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator, edge_weight=None):
    model.eval()

    out = model(features, graph, edge_weight=edge_weight)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': labels[train_mask],
        'y_pred': y_pred[train_mask],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': labels[val_mask],
        'y_pred': y_pred[val_mask],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': labels[test_mask],
        'y_pred': y_pred[test_mask],
    })['acc']

    return train_acc, valid_acc, test_acc

In [20]:
evaluator = Evaluator(name='ogbn-arxiv')

In [21]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, data.adj_t, features, train_mask, optimizer, edge_weight=edge_weights)
    result = test(model, data.adj_t, features, labels, train_mask, val_mask, test_mask, evaluator, edge_weight=edge_weights)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.0933, Train: 70.69%, Valid: 70.31% Test: 69.40%
Epoch: 200, Loss: 0.9827, Train: 73.02%, Valid: 71.74% Test: 70.63%
Epoch: 300, Loss: 0.9273, Train: 74.33%, Valid: 71.95% Test: 70.79%
Epoch: 400, Loss: 0.8858, Train: 75.28%, Valid: 72.28% Test: 71.11%
Epoch: 500, Loss: 0.8542, Train: 76.10%, Valid: 72.48% Test: 71.24%
Epoch: 600, Loss: 0.8319, Train: 76.79%, Valid: 72.77% Test: 71.52%
Epoch: 700, Loss: 0.8116, Train: 77.46%, Valid: 72.68% Test: 71.13%
Epoch: 800, Loss: 0.7914, Train: 78.00%, Valid: 72.81% Test: 71.16%
Epoch: 900, Loss: 0.7779, Train: 78.43%, Valid: 73.18% Test: 72.05%
Epoch: 1000, Loss: 0.7607, Train: 78.80%, Valid: 73.33% Test: 72.49%
Epoch: 1100, Loss: 0.7508, Train: 79.26%, Valid: 72.85% Test: 71.06%
Epoch: 1200, Loss: 0.7392, Train: 79.52%, Valid: 72.45% Test: 70.67%
Epoch: 1300, Loss: 0.7302, Train: 79.85%, Valid: 72.37% Test: 70.28%
Epoch: 1400, Loss: 0.7153, Train: 80.25%, Valid: 73.16% Test: 71.68%
Epoch: 1500, Loss: 0.7112, Train: 80.37%, V